# ZigZag API Demonstration: 
## A hardware architecture comparison demo
In this demo, we use ZigZag to optimize the mapping of AlexNet onto a TPU-like architecture.
Any ONNX model can be supplied, alongside an already implemented architecture, or a custom architecture, wich we don't get into in this first demonstration.


In [2]:
# Load in the AlexNet ONNX model. (taken from onnx model zoo)
import onnx
onnx_model_path = "alexnet.onnx"
onnx_model = onnx.load(onnx_model_path, load_external_data=False)

Next, we evaluate the hardware performance (in terms of energy (pJ) and latency (cycles)) of different accelerator architectures by calling the ZigZag API with standard-included architectures:

- TPU
- Edge TPU
- Tesla NPU
- Meta Prototype

In [3]:
from zigzag.api import get_hardware_performance
from zigzag.inputs.examples.mapping.alexnet_on_tpu_like import mapping as mapping_tpu
from zigzag.inputs.examples.mapping.alexnet_on_edge_tpu_like import mapping as mapping_edge_tpu
from zigzag.inputs.examples.mapping.alexnet_on_tesla_npu import mapping as mapping_tesla_npu
from zigzag.inputs.examples.mapping.alexnet_on_meta_prototype import mapping as mapping_meta_prototype


# List of hardware architectures we run our experiment for
hardwares = ["tpu", "edge-tpu", "tesla-npu", "meta-prototype"]
# List of mappings for each hardware (encodes the spatial dataflow)
mappings = [mapping_tpu, mapping_edge_tpu, mapping_tesla_npu, mapping_meta_prototype]

cmes = []
for (hardware, mapping) in zip(hardwares, mappings):
    # Pickle filename to save list of cmes
    pickle_filename = "outputs/list_of_cmes.pickle"
    # Call the zigzag api, using a provided accelerator and mapping
    # energy, latency = get_hardware_performance(onnx_model, accelerator, mapping)
    energy, latency, cme = get_hardware_performance(onnx_model, hardware, mapping=mapping, opt='latency', pickle_filename=pickle_filename)
    print(f"Total onnx model (energy, latency) performance = ({energy:.3e}, {latency:.3e}).")
    cmes.append(cme)

2022-11-06 23:31:49,951 - generate_layer_node_for_conv +171 - INFO - Parsed convolutional node 
2022-11-06 23:31:49,952 - generate_layer_node_for_conv +171 - INFO - Parsed convolutional node 
2022-11-06 23:31:49,953 - generate_layer_node_for_conv +171 - INFO - Parsed convolutional node 
2022-11-06 23:31:49,954 - generate_layer_node_for_conv +171 - INFO - Parsed convolutional node 
2022-11-06 23:31:49,955 - generate_layer_node_for_conv +171 - INFO - Parsed convolutional node 
2022-11-06 23:31:49,956 - generate_layer_node_for_gemm +91 - INFO - Parsed Gemm node 
2022-11-06 23:31:49,957 - generate_layer_node_for_gemm +91 - INFO - Parsed Gemm node 
2022-11-06 23:31:49,958 - generate_layer_node_for_gemm +91 - INFO - Parsed Gemm node 
2022-11-06 23:31:49,960 - parse_workload_from_onnx_model_and_mapping +101 - INFO - Created ONNXWorkload graph with 24 nodes and 23 edges.
2022-11-06 23:31:49,963 - parse_accelerator_from_path +56 - INFO - Parsed accelerator with cores [1].
2022-11-06 23:31:49,96

KeyboardInterrupt: 

### Visualizing the results
Inside the repo, we have provided a visualization script, that can automatically visualize these best "cost model evaluations" found by ZigZag.

We use the generated list of cost model evaluations (representing the total AlexNet performance for each architecture) from the previous step to visualize the energy and latency distribution across architectures.


In [ ]:
from zigzag.visualization.results.plot_cme import bar_plot_cost_model_evaluations_total

bar_plot_cost_model_evaluations_total(cmes, labels=hardwares, fig_title="Best latency mapping found", save_path="plot_total.png")